In [48]:
import os
import sys
import urllib.request
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
import time


In [36]:
from io import BytesIO
from zipfile import ZipFile
from xml.etree import ElementTree
import csv
import ssl

In [19]:
with open('data/secrets.json') as f:
    dict_api = json.load(f)
dict_dart = dict_api['dart']
api_key = dict_dart['API_KEY']

In [29]:
context = ssl._create_unverified_context()
url = f'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key={api_key}'
data = urllib.request.urlopen(url, context=context)
filename = data.info().get_filename()
with open(filename, 'wb') as f:
    f.write(data.read())
    f.close()
with ZipFile(filename, 'r') as zipObj:
   zipObj.extractall('./data/') # 현재 디렉토리에 압축을 해제

if os.path.isfile(filename):
  os.remove(filename) # 원본 압축파일 삭제

In [33]:
df_corpcodes = pd.DataFrame(columns=['corp_code', 'corp_name', 'stock_code', 'modify_date'])

tree = ElementTree.parse('./data/CORPCODE.xml')
root = tree.getroot()
for company in root.findall('list'):
    row = pd.DataFrame([[
        company.find('corp_code').text,
        company.find('corp_name').text,
        company.find('stock_code').text,
        company.find('modify_date').text
        ]], columns=df_corpcodes.columns)
    df_corpcodes = pd.concat([df_corpcodes, row], ignore_index=True)
df_corpcodes.to_csv('./data/corpcodes.csv')


In [20]:
df_corpcodes = pd.read_csv('./data/corpcodes.csv', index_col=0)

In [21]:
df_corpcodes.head()

,corp_code,corp_name,stock_code,modify_date
0,434003,다코,,20170630
1,434456,일산약품,,20170630
2,430964,굿앤엘에스,,20170630
3,432403,한라판지,,20170630
4,388953,크레디피아제이십오차유동화전문회사,,20170630


In [22]:
arr_corpcodes = df_corpcodes.corp_code.to_numpy()
arr_corpcodes

array([ 434003,  434456,  430964, ..., 1359578, 1002944, 1143737])

In [46]:
df_exist_corpcodes = pd.read_csv('./data/dart_company_base.csv')
df_exist_corpcodes = df_exist_corpcodes.drop(columns=['status', 'message', 'stock_code', 'phn_no', 'fax_no', 'hm_url', 'ir_url', 'acc_mt'])

In [30]:
arr_exist_corpcodes = df_exist_corpcodes.corp_code.to_numpy()
arr_exist_corpcodes

array([260985, 264529, 358545, ..., 186559, 476036, 899556])

In [42]:
arr_required = np.setdiff1d(arr_corpcodes, arr_exist_corpcodes)

In [43]:
len(arr_required)

93502

In [47]:
arr_corpnames = df_corpcodes.corp_name.to_numpy()
arr_exist_corpnames = df_exist_corpcodes.stock_name.to_numpy()
arr_names_required = np.setdiff1d(arr_corpnames, arr_exist_corpnames)
len(arr_names_required)

87566

In [31]:
def get_corp_num(corp_name, df_corpcodes):
    df_find = df_corpcodes[df_corpcodes['corp_name'] == corp_name]
    if len(df_find) == 0:
        return False
    else:
        return (df_find.iloc[0].corp_code)


In [10]:
get_corp_num('삼성전자', df_corpcodes)

126380

In [37]:
list_desc = []

In [38]:
epoch = 0
start = 20000 * epoch
for i, corp_code in enumerate(arr_corpcodes, start=start):
    #if corp_code in arr_exist_corpcodes:
    #    continue
    if i == 20000 + start:
        break
    time.sleep(0.067)
    url = f'https://opendart.fss.or.kr/api/company.json?crtfc_key={api_key}&corp_code={corp_code}'
    response = requests.get(url, verify=False)
    res_json = response.json()
    if int(res_json['status']) == 0:
        list_desc.append(res_json)
df_desc = pd.DataFrame(list_desc)
df_desc = df_desc.drop(columns=['status', 'message', 'stock_name', 'stock_code', 'phn_no', 'fax_no', 'hm_url', 'ir_url', 'acc_mt'])
df_desc.head()


/Users/jaegyoonoh/.pyenv/versions/3.10.1/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendart.fss.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/jaegyoonoh/.pyenv/versions/3.10.1/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendart.fss.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/jaegyoonoh/.pyenv/versions/3.10.1/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendart.fss.or.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.htm

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [39]:
len(list_desc)

0

In [51]:
res_json

{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '한종희, 경계현',
 'corp_cls': 'Y',
 'jurir_no': '1301110006246',
 'bizr_no': '1248100998',
 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.sec.co.kr',
 'ir_url': '',
 'phn_no': '031-200-1114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

In [66]:

url = f'https://opendart.fss.or.kr/api/lwstLg.json?crtfc_key={api_key}&corp_code=00164830&bgn_de=20150101&end_de=20221118'
response = requests.get(url)

res_json = response.json()
print(res_json['list'])


[{'rcept_no': '20190830001368', 'corp_cls': 'Y', 'corp_code': '00164830', 'corp_name': '한국조선해양', 'ft_ctp': '당사는 법적 절차에 따라 대응할 예정임.', 'cpct': '서울중앙지방법원', 'cfd': '2019년 08월 30일', 'icnm': '서울중앙지방법원 19카합21023 주주총회결의 효력정지 등 가처분신청서', 'ac_ap': '박근태 외 280명', 'rq_cn': '[당사자명]\n채권자 - 박근태 외 280명\n채무자 - 한국조선해양 주식회사 및 현대중공업 주식회사\n\n[항고취지]\n1. 원결정을 취소한다.\n2. 채무자 한국조선해양 주식회사가  2019. 5. 31.자 임시주주총회에서 승인한\n ‘분할계획서 승인의 건’결의의 효력을 정지한다.\n3. 채무자 한국조선해양 주식회사는 채권자와 채무자들 사이의 분할무효청구 사건의 본안 판결 확정시까지 채무자 현대중공업 주식회사가 개최하는 주주총회에서의 의결권 행사 등 일체의 주주권을 행사하여서는 아니된다.\n4. 채무자 한국조선해양 주식회사는 채권자와 채무자들 사이의 분할무효청구 사건의 본안 판결 확정시까지 일체의 본점소재지 이전행위를 하여서는 아니된다.\n5. 채무자 현대중공업 주식회사는 채권자와 채무자들 사이의 분할무효청구 사건의 \n본안 판결 확정시까지 채무자 한국조선해양 주식회사에 대하여 일체의 이익배당 행위를 하여서는 아니된다.\n6. 채무자 현대중공업 주식회사는 채무자 현대중공업 주식회사 명의로 \n금융기관으로부터 차입금을 조달하거나, 사채를 발행하여서는 아니된다.\n7. 신청비용은 채무자들이 부담한다.', 'lgd': '2019년 08월 27일'}, {'rcept_no': '20191218000287', 'corp_cls': 'Y', 'corp_code': '00164830', 'corp_name': '한국조선해양', 'ft_ctp': '당사는 법적 절차에 따라 대응할 예정임.', 'cpct': '서울고

In [67]:
df_law = pd.DataFrame(res_json['list'])
df_law.head()

,rcept_no,corp_cls,corp_code,corp_name,ft_ctp,cpct,cfd,icnm,ac_ap,rq_cn,lgd
0,20190830001368,Y,00164830,한국조선해양,당사는 법적 절차에 따라 대응할 예정임.,서울중앙지방법원,2019년 08월 30일,서울중앙지방법원 19카합21023 주주총회결의 효력정지 등 가처분신청서,박근태 외 280명,[당사자명]\n채권자 - 박근태 외 280명\n채무자 - 한국조선해양 주식회사 및 ...,2019년 08월 27일
1,20191218000287,Y,00164830,한국조선해양,당사는 법적 절차에 따라 대응할 예정임.,서울고등법원,2019년 12월 18일,서울고등법원 2019라20999 주주총회결의효력정지등 가처분,박근태 외 280명,[당사자명]\n채권자 - 박근태 외 280명\n채무자 - 한국조선해양 주식회사 및 ...,2019년 12월 18일


In [68]:
url = f'https://opendart.fss.or.kr/api/empSttus.json?crtfc_key={api_key}&corp_code=00126380&bsns_year=2015&reprt_code=11011'
response = requests.get(url)

res_json = response.json()
print(res_json['list'])
print(len(res_json['list']))
df_emp = pd.DataFrame(res_json['list'])
df_emp.head()


[{'rcept_no': '20160330003536', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'rm': '-', 'sexdstn': '남', 'fo_bbm': 'CE', 'reform_bfe_emp_co_rgllbr': '-', 'reform_bfe_emp_co_cnttk': '-', 'reform_bfe_emp_co_etc': '-', 'rgllbr_co': '13,081', 'rgllbr_abacpt_labrr_co': '-', 'cnttk_co': '76', 'cnttk_abacpt_labrr_co': '5', 'sm': '13,157', 'avrg_cnwk_sdytrn': '12.1', 'fyer_salary_totamt': '-', 'jan_salary_am': '-'}, {'rcept_no': '20160330003536', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'rm': '-', 'sexdstn': '여', 'fo_bbm': '성별합계', 'reform_bfe_emp_co_rgllbr': '-', 'reform_bfe_emp_co_cnttk': '-', 'reform_bfe_emp_co_etc': '-', 'rgllbr_co': '24,553', 'rgllbr_abacpt_labrr_co': '-', 'cnttk_co': '866', 'cnttk_abacpt_labrr_co': '785', 'sm': '25,419', 'avrg_cnwk_sdytrn': '8.5', 'fyer_salary_totamt': '1,970,979,000,000', 'jan_salary_am': '75,000,000'}, {'rcept_no': '20160330003536', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'rm': '-', 'sexdstn'

,rcept_no,corp_cls,corp_code,corp_name,rm,sexdstn,fo_bbm,reform_bfe_emp_co_rgllbr,reform_bfe_emp_co_cnttk,reform_bfe_emp_co_etc,rgllbr_co,rgllbr_abacpt_labrr_co,cnttk_co,cnttk_abacpt_labrr_co,sm,avrg_cnwk_sdytrn,fyer_salary_totamt,jan_salary_am
0,20160330003536,Y,00126380,삼성전자,-,남,CE,-,-,-,"13,081",-,76,5,"13,157",12.1,-,-
1,20160330003536,Y,00126380,삼성전자,-,여,성별합계,-,-,-,"24,553",-,866,785,"25,419",8.5,"1,970,979,000,000","75,000,000"
2,20160330003536,Y,00126380,삼성전자,-,여,CE,-,-,-,"2,556",-,213,186,"2,769",7.0,-,-
3,20160330003536,Y,00126380,삼성전자,-,남,IM,-,-,-,"19,771",-,119,5,"19,890",10.4,-,-
4,20160330003536,Y,00126380,삼성전자,-,여,IM,-,-,-,"7,391",-,209,185,"7,600",7.8,-,-


In [69]:
url = f'https://opendart.fss.or.kr/api/exctvSttus.json?crtfc_key={api_key}&corp_code=00126380&bsns_year=2015&reprt_code=11011'
response = requests.get(url)

res_json = response.json()
print(res_json['list'])
print(len(res_json['list']))
df_exctv = pd.DataFrame(res_json['list'])
df_exctv.head()

[{'rcept_no': '20160330003536', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'nm': '권오현', 'sexdstn': '남', 'birth_ym': '1952년 10월', 'ofcps': '대표이사', 'rgist_exctv_at': '등기임원', 'fte_at': '상근', 'chrg_job': '이사회 의장 \nDS부문 경영전반 총괄', 'main_career': '삼성전자\nDS부문장', 'mxmm_shrholdr_relate': '-', 'hffc_pd': '46개월', 'tenure_end_on': '2018년 03월 15일'}, {'rcept_no': '20160330003536', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'nm': '윤부근', 'sexdstn': '남', 'birth_ym': '1953년 02월', 'ofcps': '대표이사', 'rgist_exctv_at': '등기임원', 'fte_at': '상근', 'chrg_job': 'CE부문 경영전반 총괄', 'main_career': '삼성전자\nCE부문장', 'mxmm_shrholdr_relate': '-', 'hffc_pd': '34개월', 'tenure_end_on': '2016년 03월 14일'}, {'rcept_no': '20160330003536', 'corp_cls': 'Y', 'corp_code': '00126380', 'corp_name': '삼성전자', 'nm': '신종균', 'sexdstn': '남', 'birth_ym': '1956년 01월', 'ofcps': '대표이사', 'rgist_exctv_at': '등기임원', 'fte_at': '상근', 'chrg_job': 'IM부문 경영전반 총괄', 'main_career': '삼성전자\nIM부문장', 'mxmm_shrholdr_relate': '-', 

,rcept_no,corp_cls,corp_code,corp_name,nm,sexdstn,birth_ym,ofcps,rgist_exctv_at,fte_at,chrg_job,main_career,mxmm_shrholdr_relate,hffc_pd,tenure_end_on
0,20160330003536,Y,00126380,삼성전자,권오현,남,1952년 10월,대표이사,등기임원,상근,이사회 의장 \nDS부문 경영전반 총괄,삼성전자\nDS부문장,-,46개월,2018년 03월 15일
1,20160330003536,Y,00126380,삼성전자,윤부근,남,1953년 02월,대표이사,등기임원,상근,CE부문 경영전반 총괄,삼성전자\nCE부문장,-,34개월,2016년 03월 14일
2,20160330003536,Y,00126380,삼성전자,신종균,남,1956년 01월,대표이사,등기임원,상근,IM부문 경영전반 총괄,삼성전자\nIM부문장,-,34개월,2016년 03월 14일
3,20160330003536,Y,00126380,삼성전자,이상훈,남,1955년 06월,사내이사,등기임원,상근,전사 경영지원업무 총괄,삼성전자\n경영지원실장,-,34개월,2016년 03월 14일
4,20160330003536,Y,00126380,삼성전자,이인호,남,1943년 11월,사외이사,등기임원,비상근,전사 경영전반에 대한 업무,(전)신한은행 은행장,-,70개월,2016년 03월 14일
